## Generative Adversarial Network

Generative Adversarial Networks a.k.a GANs are emerging techniques that model high dimensional distribution of data. They achieve this by training a pair of networks, Generator and Discriminator, in competition with each other. As an analogy we can think of these models as an art forger and the other being an art expert. In GAN literature we term Generator as the forger and the duty of the generator is to produce fake images(forgeries) to deceive the art expert(Discriminator). The Discriminator which receives both the real images and fake images tries to distinguish between them and find the fake images. Both are trained simulataneously and are always in competition with each other. This competition between the Generator and Discriminator drives them to improve their models continuously. The models are trained until the generator produces the fake images that are indistinguishable with the real images.  <br>

In this setup, the generator do not have access to the synthetic images whereas the discriminator has access to both the real and fake images. While training the models, the supervisory signal to the discriminator is given by knowing whether the images came from a real image stack or generated image stack whereas the generator in the setup receives the error signal only from the discriminator.  

Let us define Discriminator D that takes image as input and produces a number **(0/1)** as output and a Generator G that takes random noise as input and outputs a fake image. In practise, G and D are trained alternately i.e., For a fixed generator G, the discriminator D is trained to classify the training data as real(output a value close to 1) or fake(output a value close to 0). In the next step we freeze the Discriminator assuming it as optimal and we the train the generator G to produce a image(fake) that outputs a value close to 1(real) when passed through the discriminator D. Thus, if the generator is perfectly trained then the discriminator D will be maximally confused by the images generated by G and predicts 0.5 for all the inputs. 



In this assignment, we will try to implement a Generative Adversarial Network on MNIST data and generate images that resemble the digits from the MNIST dataset.

To implement a GAN, we basically require 5 components:

- Real Dataset (real distribution)
- Low dimensional random noise that goes into the generator to produce fake images
- Generator that generates fake images
- Discriminator that acts as an expert to distinguish real and fake images.
- Training loop where the competition occurs and models better themselves


Let us implement each of the part and train the overall model:

In [1]:
## import packages

import torch
import random
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset

import matplotlib.pyplot as plt
%matplotlib inline



## Checks for the availability of GPU 
if torch.cuda.is_available():
    print("working on gpu!")
    device = 'cuda'
else:
    print("No gpu! only cpu ;)")
    device = 'cpu'
    
## The following random seeds are just for deterministic behaviour of the code and evaluation

##############################################################################
################### DO NOT MODIFY THE CODE BELOW #############################    
##############################################################################

if device == 'cpu':    
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
elif device == 'cuda':
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = '42'

############################################################################### 

No gpu! only cpu ;)


In [2]:
def preprocess_img(x):
    return 2 * x - 1.0

def deprocess_img(x):
    return (x + 1.0) / 2.0

In [3]:
import torchvision
import torchvision.transforms as transforms
import os

if not os.path.isdir('./data'):
    os.mkdir('./data')
root = './data/'

train_bs = 64

training_data = torchvision.datasets.FashionMNIST(root, train=True, transform=transforms.ToTensor(),download=True)
train_loader=torch.utils.data.DataLoader(dataset=training_data, batch_size=train_bs, shuffle=True, drop_last=True)


In GANs we input random noise through the generator to produce fake images which resemble the real distribution. Let us define a function which takes (batchsize, dimension) as input and returns a random noise of requested dimensions.

In [4]:
## bs - batchsize, dim - input dimension of random noise
## return a random noise of requested dimension from a normal distribution. 
def noise(bs, dim):
    out = torch.randn((bs, dim))
    return out
a = noise(8,15)
print(a.size())

torch.Size([8, 15])


Generator architecture:

- noise_dim -> 1024
- ReLU
- 1024 -> 1024
- ReLU
- 1024 -> 784
- TanH 
- clip image [-1, +1]

In [5]:
## In this cell we define a generator which takes in random noise as input and outputs a image of size equal
## to the images from real distribution

## we follow the same style for defining the model as worked in our previous assignments.
class Generator(nn.Module):
    def __init__(self, noise_dim=100, out_size=784):
        super(Generator, self).__init__()
        
        ### MODEL HERE ######
        ### define a fully connected layer from input_dim -> 1024
        self.layer1 = nn.Linear(noise_dim, 1024)
        self.relu = nn.ReLU()
        ### define a ReLU activation function
        
        ### define a fully connected layer from 1024 -> 1024
        self.layer2 = nn.Linear(1024, 1024)
        ### define a ReLU activation function
        self.tanh = nn.Tanh()
        ### define a fully connected layer from 1024 -> 1024
        self.layer3 = nn.Linear(1024, out_size)
        ### define a TanH activation function
        
    def forward(self, x):
        ### Make a forward pass of the model through the layers as described in the 
        ### generator architecture
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.tanh(x)
        x = torch.clamp(x, -1, 1)
        
        ### clip the individual values of the output vector such that all the values lie between [-1, +1]
        x = x.view(x.size(0), 28, 28)
        
        return x
        
        
        
        

Discriminator architecture:

- input_size->256
- LeakyReLU alpha=0.01
- 256e->256
- LeakyReLU alpha-=0.01
- 256->1

In [6]:
## Similar to the generator, we now define a discriminator which takes in a vector and output a value between 
## 0 and 1. We define the Discriminator as given in the above architecture.

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        ## define a fully connected layer from from input_dim -> 256
        self.layer1 = nn.Linear(784, 256)
        
        ### define a LeakyReLU activation function with alpha 0.01
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)
        
        ### define a fully connected layer from 256 -> 256
        self.layer2 = nn.Linear(256, 256)
        
        ### define a LeakyReLU activation function with alpha 0.01
        
        ### define a fully connected layer from 256 -> 1
        self.layer3 = nn.Linear(256, 1)
        
    def forward(self, x):
        ### Make a forward pass of the model through the layers as described in the 
        ### Discriminator architecture
        x = x.view(x.size(0), -1)
        x = self.layer1(x)
        x = self.leaky_relu(x)
        x = self.layer2(x)
        x = self.leaky_relu(x)
        x = self.layer3(x)
        
        
        return x
        
        

In [7]:
bce_loss = nn.BCEWithLogitsLoss()

In [8]:
def DLoss(logits_real, logits_fake, targets_real, targets_fake):
    d1 = bce_loss(logits_real, targets_real)
    d2 = bce_loss(logits_fake.detach(), targets_fake)
    total_loss = d1 + d2
    avg_loss = torch.mean(total_loss)
    return avg_loss
    

In [9]:
def GLoss(logits_fake, targets_real):
    g_loss = bce_loss(logits_fake, targets_real)
    avg_loss = torch.mean(g_loss)
    return avg_loss

In [10]:
noise_dim = 96
epochs = 100

generator = Generator(noise_dim)
discriminator = Discriminator()

In [11]:
## Now, we define an optimization strategy to optimize the model weights. For this assignment we will use a
## stochastic gradient descent 

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=1e-3)

In [ ]:
## Training loop

for epoch in range(epochs):
    for i, (images, _) in enumerate(train_loader):
        targets_real = torch.ones((images.size(0), 1), requires_grad=False)
        targets_fake = torch.zeros((images.size(0), 1), requires_grad=False)
        
        fake_images = generator(noise(train_bs, noise_dim))
        fake_logits = discriminator(fake_images)
        real_logits = discriminator(images)
        
        optimizer_G.zero_grad()
        generator_loss = GLoss(fake_logits, targets_real)
        generator_loss.backward()
        optimizer_G.step()
        
        optimizer_D.zero_grad()
        discriminator_loss = DLoss(real_logits, fake_logits, targets_real, targets_fake)
        discriminator_loss.backward()
        optimizer_D.step()
        
        if epoch % 5 == 0:
            fake_set = (generator(noise(train_bs, noise_dim))).detach().numpy()
            fig = plt.figure(figsize=(8,8))
            for i in np.arange(1, 13):
                ax = fig.add_subplot(3,4,i, frameon=False)
                img = fake_set[i]
                plt.imshow(img)


/home/local/ASUAD/bnagaban/anaconda3/envs/myenv/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
